<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import *
from ipywidgets.widgets import IntSlider
from ipywidgets import interact
%matplotlib inline

# Klassifikation mit Support Vector Machines

Bisher haben wir in überwachten ML-Agorithmen Daten verwendet, um zu lernen, zu welchen Klassen die Daten gehören:
- Die **logistische Regression** lernt Abhängigkeiten zwischen Eingabedaten und Zielen
- **Entscheidungsbäume** verwenden eine Hirarchiestruktur, um die Auswirkungen verschiedener Attribute auf das Ziel abzuwägen

In beiden Verfahren werden Entscheidungsgrenzen gelernt, mit denen sich die Punkte des Trainingsdatensatzes möglichst gut separieren lassen.
Der konkrete Verlauf der Entscheidungsgrenzen wird dabei nicht nicht berücksichtig.
D.h., wenn es mehrere Entscheidungsgrenzen gibt, die die Punkte bestmöglich klassifizieren, liefern die o.g. Verfahren nicht unbedingt die beste Entscheidungsgrenze.

Aber was bedeutet überhaupt *beste* Entscheidungsgrenze?
Man kann sich leicht vorstellen, dass eine Entscheidungsgrenze umso besser ist, je mehr sie durch die *Mitte* der Raumes verläuft, der zwischen den *Punktwolken* der einzelnen Klassen liegt.
Wenn die Entscheidungsgrenze so eingezogen wird, ist der Abstand zwischen den *äußeren* Punkten der Trainingsdaten und der Entscheidungsgrenze maximal groß.
Mathematisch gesprochen erzeugt ein Klassifikator, der nach diesem Prinzip funktioniert, Hyperebenen mit größtmöglicher geometrischer Marge.
Man spricht daher im Englischen auch von einem *Maximal Margin Classifier*.

Die *Support Vector Machine* (SVM) ist eine Form der *Maximal Margin Classifier*.
Mit SVMs lassen sich sowohl lineare als auch nichtlineare Klassifikationsaufgaben lösen.
Ebenfalls können SVMs zur Regression oder Ausreißer-Erkennung eingesetzt werden.
SVMs zählen zur Klasse der Instanz-basierten Lernverfahren und eignen sich besonders zur Klassifikation komplexer Datensätze (viele Merkmale) mit kleiner oder mittlerer Größe (moderate Anzahl von Datenpunkten).
Im Gegensatz zu statistischen Ansätzen (wie etwa der der logistischen Regression) basiert die SVM auf den geometrischen Eigenschaften der Daten.
Dabei ist die Bedeutung der Variablen bei der SVM weniger relevant, weswegen dieses Modell gut mit unstrukturierten und halbstrukturierten Daten wie Texten und Bildern funktioniert.


In [ ]:
n_classes = 2
n_data = 50

# generating two-class dataset
X, y = make_blobs(n_samples=n_data, centers=n_classes, random_state=1, center_box=(0, 10))
X = MinMaxScaler().fit_transform(X)
y_col = np.array(['red' if i == 0 else 'lime' for i in y])

# plotting the data according to class
for i, s, m in [(0, 100, 'o'), (1, 80, 's')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black')
plt.axis('off')
plt.show()

Die beiden Datencluster sind eindeutig linear trennbar.
Dies ist nicht immer der Fall, für den Moment gehen wir aber dvon aus, dass dies für unseren Datensatz gilt.
Wo können wir die Grenze ziehen?
In der folgenden Abbildung ziehen wir mehrere möglich Hyperebenen zwischen die Punktewolken ein.

In [ ]:
xline = np.linspace(0,1)

# plotting the data according to class
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label)

# plotting potential hyperparameters
for m, b in [(-1, 1), (0, 0.45)]:
    plt.plot(xline, m * xline + b)
plt.plot(xline*0+0.55, xline)

plt.axis('off')
plt.legend()
plt.show()

Jede der Hyperebenen separiert die *Positiven* und *Negativen* Datenpunkte optimal.
Aber nicht alle Hyperebenen sind **optimal generalisierend**.
Bei der grünen und orongenen Hyperebene ibt es *Bereiche*, die zwar nach an den Punkten einer Klasse liegen, aber dem Bereich der anderen Klasse zugeordnet sind.

Die blaue Hyperebene maximiert auch die Bereiche *um* die Punktwolken, sie schafft also die breiteste Marge (eng. *Margin*) zwischen den Punkten *am Rand* und der trennenden Hyperebene.
Daher ist sie objektiv besser als die beiden anderen eingezeichneten Möglichkeiten.

In [ ]:
xline = np.linspace(0, 1)
yline = -1.5*xline+1.3
delta = 0.35

# plotting the data according to class
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label)

# plotting decision boundry
for s, m in [(1, 'k--'),(0, 'k'),(-1, 'k--')]:
    plt.plot(xline, yline +delta*s, 'k--', alpha = 0.8)

# Annotationen
plt.text(0.26, 0.64, s='Trennende\nHyperebene\n',fontsize= 13, rotation = -50)
plt.annotate(text='', xy=(xline[30], yline[30]), xytext=(xline[36],yline[36]+delta), arrowprops=dict(arrowstyle='<->'))
plt.text(0.7, 0.3, s='Marge',fontsize= 13, rotation = -45)

plt.xlim(0,1)
plt.ylim(0,1)
plt.axis('off')
plt.legend();

**SVM Klassifizierer** finden Hyperebenen mit maximaler Marge, sind also als *Maximal Margin Classifier* einzustufen.
Um eine *Support Vector Machine* mit `sklearn` zu erstellen, benötigen wir ein Objekt vom Typ `SVM`. Dieses Modell trainieren wir auf den entsprechenden Datensatz mit der `fit`-Funktion:

In [ ]:
model = SVC(kernel='linear', C=1e5).fit(X, y)

Nun können wir die gefunden Hyperebenen darstellen:

In [ ]:
# plotting data
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label)

# support vectors
plt.scatter(model.support_vectors_[:, 0],
           model.support_vectors_[:, 1],
           s=400, edgecolors='black', facecolors='none')

# create grid to evaluate model
xx = np.linspace(0, 1, 30)
yy = np.linspace(0, 1, 30)
yy, xx = np.meshgrid(yy, xx)
xy = np.vstack([xx.ravel(), yy.ravel()]).T
P = model.decision_function(xy).reshape(xx.shape)

# plotting decision boundry
plt.contour(xx, yy, P, colors='k', levels=[-1, 0, 1],
            alpha=0.8, linestyles=['--', '-', '--'])


plt.axis('off')
plt.legend()
plt.show()

Wo die Hyperebene mit der maximalen Marge liegt, wird von den Punkten am äußeren *Rand* der Punktwolken bestimmt (in der Abbildung zu erkennen durch die schwarze Umrandung).
Diese Punkte nennt man auch **Support Vektoren**.
Wie sich leicht erkennen lässt, haben diese Punkte eine herausragende Bedeutung für SVMs.
Im Folgenden wollen daher betrachten, wie man diese Punkte identifizieren kann.

## Support Vector Machines

Wenn wir von einem linearen Modell ausgehen, lässt sich unsere Hyperebene im $n$-dimensionalen Raum mit der Gleichung
$$\langle\textbf{w},\textbf{x}\rangle+b=0$$
beschreiben.

Dabei ist $\langle \textbf{w}, \textbf{x} \rangle$ das Standardskalarprodukt des Paramter-Vektors $\mathbf{w}$ und des Eingabe-Vektors $\mathbf{x}$. Wenn wir davon ausgehen, dass $\textbf{w}$ und $\textbf{x}$ Spaltenvektoren sind, erhalten wir das Skalarprodukt, wenn wir $\textbf{w}$ durch Transponieren zum Zeilenvektor machen und dann die generelle Matrizenmultiplkation anwenden. Die obige Gleichung ist also identisch mit der Form $ \textbf{w}^T\textbf{x}+b$



Eine solche Hypereben trent den $n$-dimensionalen Raum in zwei Halbräume.
Auf welcher Seite, also in welchem Halbraum ein Punkt mit dem Ortsvektor $\textbf{x}_i$ liegt, erkennt man durch einsetzen in die Gleichung. Aller Punkte $\textbf{x}_i$, mit
- $\textbf{w}^T\textbf{x}_i+b<0$ liegen in der ersten Halbebene
- $\textbf{w}^T\textbf{x}_i+b=0$ liegen auf der Ebene
- $\textbf{w}^T\textbf{x}_i+b>0$ liegen in der zweiten Halbebene

Nehmen wir an, dass die Punkte in der ersten Halbebene die Label $y_i=-1$ tragen und die Punkte auf der zweiten Halbebene die Label $y_i=1$, so gilt:
 
$$y_i(\textbf{w}^T\textbf{x}_i+b)>0$$

Wenn wir also die Vorzeicheinfunktion verwenden erhalten wir auf diesem Weg einen Klassifizierer, der die Punkte im Raum in die Klassen $-1$ (Negative) und $1$ (Positive) aufteilt:
$$ \hat{y}=sign(\textbf{w}^T\textbf{x}_i+b)$$



In [ ]:
# Daten
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label, alpha=0.2)

# Entscheidungsgrenze
plt.contour(xx, yy, P, colors='k', levels= 0,
            alpha=0.8, linestyles='-')

# Normalenvektor w und Bias b
for xy, xyt, st, lxy, l in [((0,0), (0.42,0.68), '<->', (0.15, 0.4), '$b$'),
#                           ((0.52, 0.52), (0.57,0.6), '<|-', (0.5, 0.6), '$\overrightarrow w$'),
                            ((0.52, 0.52), (0.905,1.135), '<|-', (0.5, 0.6), '$\overrightarrow w$'),
                           ((0.52, 0.52), X[28], '<|-', (0.62, 0.48), '$\overrightarrow x_{neg}$')]:
    plt.annotate(text='', xy=xy, xytext=xyt, arrowprops=dict(arrowstyle=st))
    plt.text(lxy[0], lxy[1], s=l,fontsize= 14)


plt.axis('on')
plt.xlim([0, 1.3])
plt.ylim([0, 1.3])
plt.legend()
plt.show()

Da sich es sich im Folgenden mit einer *Größer-Bedingung* nicht so einfach rechnen lässt, wandeln wir die Abstandsgleichung so um, dass für alle Punkte ein Mindestabstand zur trennenden Hyperebene gefordert wird.
Diesen Abstand können wir auf $1$ setzen, um die mathematischen Voraussetzungen zu vereinfachen.
Hierbei handelt es sich lediglich um eine Art Normierung, denn jeder andere Abstandswert würde sich allein durch Skalierung der Parameter $\textbf{w}$ und $b$ erreichen lassen.
Wir können unsere Forderung also umschreiben zu:
$$
\begin{array}{ll}
    \textbf{w}^T\textbf{x}_i + b \geq +1, & y_i = +1 (Positive)\\
    \textbf{w}^T\textbf{x}_i + b \leq -1, & y_i = -1 (Negative)
\end{array}
$$

oder zusammengefasst als:

$$y_i(\textbf{w}^T\textbf{x}_i+b)\geq1$$

Mit dieser Forderung erhält man nun 2 weitere Hyperebenen $H_1$ und $H_2$, die parallel zur Entscheidungsgrenze stehen:

$$
\begin{array}{lll}
H_1 = &\textbf{w}^T\textbf{x}_i + b = +1, & \text{Hyperebene in der Klasse mit } y_i = +1 (Positive)\\
H_2 = &\textbf{w}^T\textbf{x}_i + b = -1, & \text{Hyperebene in der Klasse mit } y_i = -1 (Negative)
\end{array}
$$

In [ ]:
# plotting data
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label, alpha=0.2)

# plotting decision boundry
plt.contour(xx, yy, P, colors='k', levels= [-1 ,0, 1],
            alpha=0.8, linestyles=['--', '-', '--'])

# Plotting wieght and bias
for xy, xyt, st, lxy, l in [((0,0), (0.42,0.68), '<->', (0.15, 0.4), '$b$'),
#                           ((0.52, 0.52), (0.57,0.6), '<|-', (0.5, 0.6), '$\overrightarrow w$'),
                           ((0.52, 0.52), (0.905,1.135), '<|-', (0.5, 0.6), '$\overrightarrow w$'),
                           ((0.52, 0.52), X[28], '<|-', (0.62, 0.48), '$\overrightarrow x_{neg}$')]:
    plt.annotate(text='', xy=xy, xytext=xyt, arrowprops=dict(arrowstyle=st))
    plt.text(lxy[0], lxy[1], s=l,fontsize= 14)

plt.axis('on')
plt.xlim([0, 1.3])
plt.ylim([0, 1.3])
plt.legend()
plt.show()

Dieser Midnestabstand zur Entscheidungsgrenze impliziert, dass kein Punkt innerhalb der Marge zwischen $H_1$ und $H_2$ liegt.
Die meisten der Datenpunkte liegen abseits dieser Hyperebenen innerhalb der Halbräume.
Die Datenpunkt, die genau auf den Hyperebenen $H_1$ und $H_2$ liegen, nennt man **Support Vektoren**.
Der Abstand $\delta$  der Hyperplanes $H_1$ und $H_2$ von der Entscheidungsgrenze kann man durch den Abstand der Support Vektoren ausdrücken:
$$2\delta=(\textbf{x}_{pos}-\textbf{x}_{neg})\cdot\frac{\textbf{w}}{|\textbf{w}|}$$
$$2\delta=\frac{\textbf{w}^T \textbf{x}_{pos} - \textbf{w}^T \textbf{x}_{neg}}{|\textbf{w}|}$$
$$2\delta=\frac{(b+1)-(b-1)}{|\textbf{w}|}$$
$$2\delta=\frac{2}{|\textbf{w}|} \implies \delta=\frac{1}{|\textbf{w}|}$$

In [ ]:
# plotting data
for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
    plt.scatter(X[y==i,0], X[y==i,1], c=y_col[y==i], s=s, marker = m, edgecolors='black', label=label, alpha=0.2)
for i, m, c in [(0, 'o', 'red'),(1, 's', 'lime')]:
    plt.scatter(model.support_vectors_[i, 0],
               model.support_vectors_[i, 1],marker = m,
               s=200, edgecolors='black', facecolors=c)


# plotting decision boundry
plt.contour(xx, yy, P, colors='k', levels= [-1 ,0, 1],
            alpha=0.8, linestyles=['--', '-', '--'])

# annotation
for xy, i, lxy in [((0.47,0.61), 1, (0.35, 0.55)),
                    ((0.55,0.46), 0, (0.62, 0.5))]:
    plt.annotate(text='', xy=xy, xytext=model.support_vectors_[i], arrowprops=dict(arrowstyle='<|-|>'))
    plt.text(lxy[0], lxy[1], s='$\delta$',fontsize= 14, rotation=45)

plt.xlim(0.1,0.9)
plt.ylim(0.1,0.9)
plt.axis('off')
plt.legend()
plt.show()

Um $\delta=\frac{1}{|\textbf{w}|}$ zu maximieren, muss $|\textbf{w}|$ minimiert werden. 
Auch hier wenden wir einen kleinen Trick an.
Da sich quadratische Funktionen leichter optimieren lassen, bestimmen wir äquivalent das Minimum von $\frac{1}{2}|\textbf{w}|^2$.
Es ergibt sich also das Minimierungsproblem:
$$
\text{min: }J(\textbf{w},b) = \frac{1}{2}|\textbf{w}|^2\\\text{wobei gleichzeitig für alle } \textbf{x}_i \text{ auch } y_i(\textbf{w}^T\textbf{x}_i + b) \geq 1 \text{ gelten muss.}
$$

Zur Minimierung einer Funktion mit Nebenbedingungen kann die **Langrange-Methode** verwednet werden.
Dazu stellen wir die folgende allgemeine **Lagrange-Funktion**:
$$\mathcal{L}(\textbf{w},b,\alpha)=\frac{1}{2}|\textbf{w}|^2-\sum_{i=1}^l\alpha_i[y_i(\textbf{w}\ \textbf{x_i} + b) - 1]; \ \ \alpha_i \geq 0$$
Die partiellen Ableitungen von $L$ müssen nun gebildet und auf Null gesetzt werden, um die möglichen Extremstellen zu bestimmen:

$$\frac{\partial \mathcal{L}}{\partial \textbf{w}} = \textbf{w} - \sum_{i=1}^l\alpha_iy_i\textbf{x_i}$$

$$\frac{\partial \mathcal{L}}{\partial b} = -\sum_{i=1}^l\alpha_iy_i$$

Für die möglichen Minima der Funktion muss also gelten:

$$\textbf{w} = \sum_{i=1}^l\hat{\alpha_i}y_i\textbf{x_i} \ \text{ und } \sum_{i=1}^l\alpha_iy_i=0$$

Setzt man diese Bedingungen in die ursprüngliche Lagrange-Funktion ein, so erhält man folgenden Term:

$$\mathcal{L}(\hat{\textbf{w}},\hat{b},\alpha)=\frac{1}{2}  \sum_{i=1}^l\sum_{j=1}^l\alpha_i\alpha_jy_iy_j\textbf{w_i}^T\textbf{w_j} - \sum_{i=1}^l \alpha_i$$
$$ \text{mit } \alpha_i \geq 0 \text{ für alle } i=1,2,\dots ,l$$

Über diese Gleichung kann ein $\hat{\alpha}$ gefunden werden, dass die Funktion $\mathcal{L}$ minimiert. Sobald man $\hat{\alpha}$ kennt, kann man das Ergebnis in die Gleichung

$$\hat{\textbf{w}} = \sum_{i=1}^l\hat{\alpha_i}y_i\textbf{x_i}$$
Über einen gefunden Stützvektor kann dann den *Bias-Term* $\hat{b}$ bestimmt werden.

 Die Minimierung über die Lagrange Funktion ist mathematisch recht komplex. Wir können aber folgende Erkenntnisse festhalten:
- Das Lösen des Optimierungsproblems basiert allein auf der Berechnung von Skalarprodukten der Form $\textbf{x_i}\ \textbf{x_j}$
- Das Schätzen der Klasse eines neuen Datenpunktes $\textbf{u}$ erfolgt über die Funktion $\text{sgn}[\hat{\textbf{w}}^T \textbf{u} + \hat{b}]$

In [ ]:
def plt_a(a,size):
    # fitting svc model
    X_slice, y_slice, y_col_slice = X[:size], y[:size], y_col[:size]
    model = SVC(kernel='linear', C=a).fit(X_slice, y_slice)

    # plotting data
    fig, (ax1, ax2) = plt.subplots(ncols=2,figsize=(6, 3.5), gridspec_kw={'width_ratios':[4, 1]})
    for i, s, m, label in [(0, 100, 'o', 'Negative'), (1, 80, 's', 'Positive')]:
        ax1.scatter(X_slice[y_slice==i,0], X_slice[y_slice==i,1], c=y_col_slice[y_slice==i],
                    s=s, marker = m, edgecolors='black', label=label)
    ax1.scatter(model.support_vectors_[:, 0],
           model.support_vectors_[:, 1],
           s=400, edgecolors='black', facecolors='none')
    
    # create grid to evaluate model
    xx = np.linspace(0, 1, 30)
    yy = np.linspace(0, 1, 30)
    yy, xx = np.meshgrid(yy, xx)
    xy = np.vstack([xx.ravel(), yy.ravel()]).T
    P = model.decision_function(xy).reshape(xx.shape)

    # plotting decision boundry
    ax1.contour(xx, yy, P, colors='k', levels=[-1, 0, 1],
                alpha=0.8, linestyles=['--', '-', '--'])
    
    ax1.axis('off')
    ax1.legend()
    
    # bar graph for lagrangian multipliers
    a_list = ['a'+str(a) for a in range(model.dual_coef_.size)]
    ax2.barh(list(a_list), np.abs(model.dual_coef_).squeeze(), color='yellow')
    ax2.scatter(X[0], X[1])
    
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    
    plt.show()
    
interact(plt_a, a= IntSlider(value=20, min=1, max=20, step=1) , 
                 size= IntSlider(value=50, min=20, max=50, step=5) )

## Quellen:
[1] M. Berthold, C. Borgelt, F. Höppner and F. Klawonn, *Guide to Intelligent Data Analysis*, Springer, 2010.\
[2] Jake VanderPlas, [*Python Data Science Handbook*](https://jakevdp.github.io/PythonDataScienceHandbook), O'Reilly, 2016.\
[3] Patrick Winston. *6.034 Artificial Intelligence*. MIT OpenCourseWare, 2010, https://ocw.mit.edu. License: Creative Commons BY-NC-SA.

### Web-Quellen:
1. [Understanding the mathematics behind Support Vector Machines](https://shuzhanfan.github.io/2018/05/understanding-mathematics-behind-support-vector-machines/)
2. [Support vector machine](https://en.wikipedia.org/wiki/Support_vector_machine)